In [1]:
from IPython.display import display
import spotipy
import spotipy.util as util


client_id = 'b69a9985fa8842deb0691b2d0e3f0b69'
client_secret = 'd9e9ae2924174c139a5a9ccb303f9f3a'
redirect_uri = 'http://localhost/'

username = '22mrmbu7oumkrb56tcsclawdi'

token = util.prompt_for_user_token(username, 'user-library-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)
tracks = []

offset = 0
while True:
    track_set = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
    if track_set:
        tracks += track_set
    else:
        break

    offset += 50
print("Tracks fetched")

artist_tracks = {}
artist_related = {}
for i in range(len(tracks)):
    tracks[i] = tracks[i]['track']
    track_id = tracks[i]['id']
    artist_id = tracks[i]['artists'][0]['id']
    if artist_id not in artist_tracks:
        artist_tracks[artist_id] = [track_id]
    else:
        artist_tracks[artist_id].append(track_id)
    if artist_id not in artist_related:
        related_list = sp.artist_related_artists(artist_id)['artists']
        artist_related[artist_id] = []
        for related in related_list:
            artist_related[artist_id].append(related['id'])
    if i % 500 == 0:
        print('%d/%d tracks processed' % (i, len(tracks)))
print("Artists fetched")

Tracks fetched
0/3781 tracks processed
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
500/3781 tracks processed
retrying ...1secs
retrying ...1secs
retrying ...1secs
1000/3781 tracks processed
retrying ...1secs
retrying ...1secs
retrying ...1secs
1500/3781 tracks processed
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
2000/3781 tracks processed
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
2500/3781 tracks processed
retrying ...1secs
retrying ...1secs
3000/3781 tracks processed
retrying ...1secs
retrying ...2secs
retrying ...1secs
3500/3781 tracks processed
retrying ...1secs
retrying ...1secs
Artists fetched


In [2]:
import networkx as nx

def chunks(n, x):
    return [x[i:i + n] for i in range(0, len(x), n)]

artist_genres = {}
for chunk in chunks(50, list(artist_tracks.keys())):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']
print("Genres fetched")
        
G = nx.Graph()
for artist in artist_related:
    for related_artist in artist_related[artist]:
        G.add_edge(artist, related_artist)

Genres fetched


In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

start = datetime.now()
distance = list(nx.all_pairs_shortest_path_length(G))
print(datetime.now() - start)

start = datetime.now()
distance = list(nx.shortest_path_length(G))
print(datetime.now() - start)

start = datetime.now()
distance = list(nx.all_pairs_bellman_ford_path_length(G))
print(datetime.now() - start)

start = datetime.now()
distance = list(nx.all_pairs_dijkstra_path_length(G))
print(datetime.now() - start)

# distance = nx.floyd_warshall_numpy(G)
# similarity = np.exp(-distance / distance.std())

0:08:51.385593
0:08:30.749811
0:18:31.135313
0:08:56.980165


NetworkXError: Graph is not weighted.

In [13]:
i = 0
for dist in distance:
    print(dist)
    print('\n')
    i += 1
    if i > 10:
        break

('3mVwDyQRNVMy3ZNuTabf6f', {'3mVwDyQRNVMy3ZNuTabf6f': 0, '0gXx2aQ2mfovDfqCw10MQC': 1, '532VhOtZQ47TWSTGwbZ4kF': 1, '6Ny7ldBgUotuALx5hSFi4e': 1, '0Y0QSi6lz1bPik5Ffjr8sd': 1, '2fNr4ldujwq97v1jWeqs8K': 1, '4yimrbAH3tCaYHxuwksfY6': 1, '0uOUEsjCUSv2YvZmZtmBsX': 1, '5ic5UHDgniVG1xKLAIU9sG': 1, '7oB4pCSzvvRtZka6DxXUfN': 1, '1521R3ksLyQyFeqdtaSZUZ': 1, '6HFxqZ6WaTb7TQUhNsK25o': 1, '2vdEGASQQlLcmOqlQhYWlN': 1, '4Ok1Cm5YX5StCQZgH0r2xF': 1, '5AoXSiM511L4To0AN5oZVB': 1, '4BwM43tn51Mq0Dn8NMB5zk': 1, '3qNEgvMVFSPWLeBGuqRQ4F': 1, '2c2X6dr1PHctH24BxNOjHi': 1, '1RxjZjOSbva0JR7T2LsRve': 1, '59DcOmdlmKUgsUrb2S7Nb6': 1, '1XXHaDxpaVojtY2QKYaQzl': 1, '6AKWjx5wmOI3bCeLrxFjgv': 1, '4E69riquObrTmBTVuF1b7B': 1, '4OHaWpWPKDcZgOOUn9elFn': 1, '2tW7TitNzNibILuWCEhBLS': 1, '3hZYcnuncZYCJzqO9pwpAK': 1, '4B7EVtenNIwOUxmQ5o5GNv': 1, '3OicELscJlOf5XnrApq6ur': 1, '3zz44G0jfFiUGVCvad9GSL': 1, '7CCrWKLbAto1EXI2eoG9Vu': 1, '1LTDQMVIIdgiJePJvUkBA3': 1, '7ERTOmWXRyme5WJUpXVVSl': 1, '0w2QFhEnPG6tLIvkogHbUi': 1, '60lZ2sr01yS7uJ

In [17]:
import sklearn.cluster

clustering = sklearn.cluster.SpectralClustering(n_clusters=5, affinity='precomputed').fit_predict(similarity)

artist_cluster = {}
nodes = list(G.nodes())
for i in range(len(nodes)):
    artist_cluster[nodes[i]] = clustering[i]

<class 'list'>


In [18]:
'artists' in nodes

True

In [73]:
counts = {'cluster': [], 'genre': [], 'count': []}
for cluster in df['cluster'].unique():
    df_cluster = df[ df['cluster']==cluster ]
    genre_count = dict()
    for genres in df_cluster['genres']:
        for genre in genres:
            if genre in genre_count:
                genre_count[genre] += 1
            else:
                genre_count[genre] = 1
    for genre, count in genre_count.items():
        counts['cluster'].append(cluster)
        counts['genre'].append(genre)
        counts['count'].append(count)

counts = pd.DataFrame(counts)
counts.sort_values('count', ascending=False, inplace=True)
original_counts = counts.copy()

clusters_to_name = set(counts['cluster'].unique())
playlists = dict()
while clusters_to_name:
    cluster = counts.iloc[0]['cluster']
    name = counts.iloc[0]['genre']
    playlists[name] = cluster
    counts = counts[ (counts['cluster']!=cluster) & (counts['genre']!=name) ]
    clusters_to_name.remove(cluster)
    
playlist_tracks = dict()
from collections import OrderedDict
for name in playlists:
    playlist_tracks[name] = df[ df['cluster']==playlists[name] ].index.values
    playlist_genres = original_counts[ original_counts['cluster']==playlists[name] ].sort_values('count', ascending=False)['genre'].values[:10]
    playlist_counts = original_counts[ original_counts['cluster']==playlists[name] ].sort_values('count', ascending=False)['count'].values[:10]
    playlist_counts = list(map(int, playlist_counts))
    playlists[name] = OrderedDict(zip(playlist_genres, playlist_counts))

In [70]:
token = util.prompt_for_user_token(username, 'playlist-modify-private', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

for name in playlist_tracks:
    id = sp.user_playlist_create(username, 'sortify '+name, public=False)['id']
    for chunk in chunks(100, playlist_tracks[name]):
        sp.user_playlist_add_tracks(username, id, chunk)

In [ ]:
classified = [[] for i in range(clusters)]
ids = df.index.values
for i in range(len(df)):
    cluster = model.predict(X[0].reshape(1, -1))[0]
    classified[cluster].append(ids[i])
for i in classified:
    print(len(i))

In [75]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(playlists)

{   'edm': OrderedDict([   ('edm', 433),
                           ('electro house', 412),
                           ('brostep', 260),
                           ('electronic trap', 234),
                           ('big room', 194),
                           ('progressive house', 171),
                           ('tropical house', 166),
                           ('progressive electro house', 158),
                           ('pop', 154),
                           ('bass trap', 148)]),
    'electro house': OrderedDict([   ('electro house', 218),
                                     ('edm', 185),
                                     ('electronic trap', 120),
                                     ('tropical house', 114),
                                     ('brostep', 113),
                                     ('house', 111),
                                     ('pop', 84),
                                     ('big room', 84),
                                     ('progressive hou